# ESG/サステナビリティ分析 ハンズオン

このノートブックでは、Snowflake Cortex AIを使用してPDFからテキストを抽出し、セマンティック検索とAIエージェントを構築します。

## 前提条件
- `setup.sql` でセッション1（環境構築）が完了していること
- ステージにPDFファイルがコピーされていること

## ハンズオン構成
1. AISQLを理解する
2. データ準備（AI_PARSE_DOCUMENT & チャンク化）
3. Cortex Search の設定
4. Cortex Agent の設定

![img](https://lh3.googleusercontent.com/pw/AP1GczMrPK0TkkHHWw_f1rPigAoYqtVYz4RI1FyKGBDnjFLUnJ5I9bNRL4eZYYIMdZzscttE4OBPWOrXpcM8oSGgN1LgNrM8SwTup8IQ8QoLTandw0Zo1Jl4uS0T8qO3LMUrnqWdx41et2IB4WOCRmIVGG0g=w1642-h924-s-no-gm?authuser=0)

## 📦 接続設定


In [ ]:
-- 接続情報の設定
USE ROLE ACCOUNTADMIN;
USE DATABASE DEMO_DB;
USE SCHEMA DEMO_SUSTAINABILITY;
USE WAREHOUSE HANDSON_WH;

In [ ]:
-- ステージ内のファイル確認
LIST @document_stage;

# セッション1: AISQLを理解する
![img](https://lh3.googleusercontent.com/pw/AP1GczNSO-h_-_0ggsGkzWfC3idPcaV5pv52SIixGW6w6YjC9kYveqnIGMlqeUEFAykwp3bxDL54UhKVnxLFjFvo9v16rh-aqIqF6MUh-WxJmOeesyT6bM6sbwW-aRXqXgpuDNmjFrxY3thax72fMO0t70eE=w1647-h924-s-no-gm?authuser=0)

### SQLでAI_COMPLETE関数と試してみる

In [ ]:
-- SQL の場合
SELECT AI_COMPLETE('claude-4-sonnet', 'Snowflakeの特徴を50字以内で教えてください。') AS RESPONSE;

### Python で AI_COMPLETE 関数を試す

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import ai_complete 

session = get_active_session()
df = session.range(1).select(
    ai_complete('snowflake-arctic', 'Snowflakeの特徴を50字以内で教えてください。').alias('response')
)
df

### AI_CLASSIFY関数によるマルチラベル分類

In [ ]:
SELECT 
    AI_CLASSIFY(
    '当社の子会社であるA工場において、地域の環境規制値を超える排水が河川に流出していた事実が判明しました。原因は浄化装置の故障を現場判断で放置していたことにあります。地域住民への謝罪とともに、再発防止策を策定中です。', -- 分類するテキスト
    ['Environment (環境)', 'Social (社会)', 'Governance (ガバナンス)', 'Other'] -- 分類カテゴルのリスト
) AS DETECTED_CATEGORY;

-- 期待される結果: {"label": "Environment (環境)"} 
-- ※モデルによってはGovernanceも高く出る可能性があります（不祥事対応のため）

### AI_FILTER関数でグリーンウォッシュのフィルタリング
環境に配慮します」といった定性的な（ふわっとした）PR文言を除外し、数値や具体的な投資額が記載されている「実質的な」レコードだけを抽出

In [ ]:
WITH ESG_DATA AS (
    -- サンプルデータ：混在する「ふわっとした宣言」と「具体的な報告」
    SELECT 
        COLUMN1 AS COMPANY_NAME, 
        COLUMN2 AS REPORT_TEXT 
    FROM VALUES
    ('Company A', '当社は地球環境との共生を最優先課題とし、サステナブルな未来の実現に向けて全社一丸となって努力を続けてまいります。'),
    ('Company B', '2023年度は、再生可能エネルギー導入に50億円を投資し、自社工場の電力の40%を太陽光発電に切り替えました。これによりScope2排出量を前年比15%削減しました。'),
    ('Company C', 'ダイバーシティ＆インクルージョンは我々のDNAであり、すべての従業員が輝ける職場環境を提供することを約束します。'),
    ('Company D', '女性管理職比率を2025年までに30%にする目標に対し、現在は24%です。今年度はメンター制度の導入により、新たに15名の女性管理職が誕生しました。')
)

SELECT 
    COMPANY_NAME,
    REPORT_TEXT
FROM ESG_DATA
WHERE 
    -- 【AI_FILTER関数の核心部分】
    -- テキストに「具体的な数値」や「実績」が含まれる場合のみ TRUE を返させる
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-70b', 
        '以下のテキストを分析してください。このテキストには、具体的な「数値データ（金額、％、人数など）」や「具体的な行動実績」が含まれていますか？
         単なる精神論やスローガンである場合は No、具体的な証拠がある場合は Yes とだけ答えてください。
         
         テキスト: ' || REPORT_TEXT
    ) ILIKE '%Yes%'; -- AIが Yes と答えた行だけを残す

# セッション2: データ準備（AI_PARSE_DOCUMENT & チャンク化）

## Step 2-1: 運用機関サステナビリティレポートの処理

![img](https://lh3.googleusercontent.com/pw/AP1GczOjH2otlQpQbjuiRW2FyDe6Mk6dy-WAjxhJuH7S0xaaThg5WPaRD1KwoLc30FP7zoCLDFRiArVLgsdinh26SA2jYNHkU-BV2z3SSlUW3PItf8_yYqs1bs6wznj50JXJpS-8P2g_Jp2IPDUKXjYcaoG1=w1637-h917-s-no-gm?authuser=0)

### AI_PARSE_DOCUMENTとは？
`AI_PARSE_DOCUMENT`は、Snowflake Cortex AIが提供するPDF解析機能です。

**主なパラメータ:**
- `mode`: `'LAYOUT'`（レイアウト維持）または `'OCR'`（画像からのテキスト抽出）
- `page_split`: `true`（ページごとに分割）または `false`（1つのドキュメントとして処理）


### 運用会社のESGレポートを構造化データへと変換
三井住友トラストアセットマネジメントの[サンプルデータ](https://www.smtam.jp/company/sustainability/) を利用

![img](https://lh3.googleusercontent.com/pw/AP1GczO7t2oaEXvW4kkkpnIzBMR3ixndnJyrFwDc71Bjk8fZ5FIGNRwnUj70JUYz1pjnkuQiTxZRKGDHhOqKe7A-xTR1sWoQqYslg9Fbi688o1tGM72iiVp723JRqzED5_-aEKIuo1dcV7oIgI9i43VsWsxp=w1306-h924-s-no-gm?authuser=0)

In [ ]:
-- 15分くらい時間がかかる可能性もあり
-- PDFからテキストを抽出（AI_PARSE_DOCUMENT使用）
CREATE OR REPLACE TABLE am_sustainability_report AS
SELECT
    relative_path,
    GET_PRESIGNED_URL('@demo_db.demo_sustainability.document_stage', relative_path) AS scoped_file_url,
    AI_PARSE_DOCUMENT(
        TO_FILE('@demo_db.demo_sustainability.document_stage', relative_path), 
        {'mode': 'LAYOUT', 'page_split': true}
    ) AS raw_text_dict,
    raw_text_dict:content::string AS raw_text
FROM DIRECTORY('@demo_db.demo_sustainability.document_stage')
WHERE relative_path LIKE 'am_esg_report/%.pdf';

In [ ]:
-- 結果確認
SELECT * FROM am_sustainability_report;

### SPLIT_TEXT_RECURSIVE_CHARACTERとは？
長いテキストをRAG（検索拡張生成）用に適切なサイズに分割する関数です。

![img](https://lh3.googleusercontent.com/pw/AP1GczPZcCvHPhKuINu1hPD1pODZZWdQ3KW_MzPeQVv6rO0vmnlNcp9K8tir3r7YZJtEcd4aqeypQR5BRU_coUjQMG0er-H57QGJvfJhtoIBj3iHQLRGwIJYFrXcsPaLCsmO-94v-qX7qSP59U42diYfvaVK=w1637-h917-s-no-gm?authuser=0)

**パラメータ:**
- 第1引数: 分割対象のテキスト
- 第2引数: フォーマット（`'markdown'`, `'plain'`など）
- 第3引数: 最大チャンクサイズ（文字数）
- 第4引数: オーバーラップサイズ（文脈維持のため）


本ハンズオンでは以下を設定
- 分割対象: contentカラムのテキストを分割
- フォーマット: 'markdown' - ヘッダー、コードブロック、テーブルで分割
- チャンクサイズ: 1000文字 - 各チャンクの最大文字数
- 重複サイズ: 100文字 - 隣接するチャンク間で重複させる文字数
- 処理結果: 1000文字以下のチャンクに分割され、100文字ずつ重複

[ドキュメント](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) (最新の情報は英語ドキュメントを参照)

In [ ]:
-- チャンク化（SPLIT_TEXT_RECURSIVE_CHARACTER使用）
CREATE OR REPLACE TABLE am_sustainability_report_chunk AS
SELECT
    t.relative_path,
    t.scoped_file_url,
    SPLIT_PART(t.relative_path, '/', -1) AS file_name,
    p.value:index::INT AS page_index, 
    c.index::INT AS chunk_index_on_page,
    c.value::STRING AS chunk_text,
    MD5_HEX(t.relative_path || ':' || p.value:index::INT || ':' || c.index::INT)::STRING AS chunk_id
FROM 
    am_sustainability_report t,
    LATERAL FLATTEN(input => t.raw_text_dict:pages) p,
    LATERAL FLATTEN(
        INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            p.value:content::STRING,
            'markdown',
            1000,
            100
        )
    ) c;

In [ ]:
-- 結果確認
SELECT * FROM am_sustainability_report_chunk;

## Step 2-2: スチュワードシップ活動原則の処理

[ソース](https://www.gpif.go.jp/esg-stw/stewardship/)データ

![img](https://lh3.googleusercontent.com/pw/AP1GczM8QM76rHWcYETJ57bLuvWb34x7motTdQixZGRokIu3JG-wbQStF2jydo1aQ3tkY27V0f49s0jM8sMbQBtMgSssBFXaCTl4YAvKhXYTqfoJtWtq4Mrf308BwizPayRdYT4S1U0kyqYN7X8fDs4s8tW4=w714-h924-s-no-gm?authuser=0)

In [ ]:
-- PDFからテキストを抽出
CREATE OR REPLACE TABLE gpif_stewardship_2025 AS
SELECT
    relative_path,
    GET_PRESIGNED_URL('@demo_db.demo_sustainability.document_stage', relative_path) AS scoped_file_url,
    AI_PARSE_DOCUMENT(
        TO_FILE('@demo_db.demo_sustainability.document_stage', relative_path), 
        {'mode': 'LAYOUT', 'page_split': false}
    ) AS raw_text_dict,
    raw_text_dict:content::string AS raw_text
FROM DIRECTORY('@demo_db.demo_sustainability.document_stage')
WHERE relative_path LIKE 'stewardship_principles/%.pdf';

### SPLIT_TEXT_MARKDOWN_HEADERとは？
Markdownのヘッダー構造を維持しながらテキストを分割する関数です。
セクションの途中で分割されることを防ぎ、文脈の「泣き別れ」を防止します。

In [ ]:
-- チャンク化（SPLIT_TEXT_MARKDOWN_HEADER使用）
CREATE OR REPLACE TABLE gpif_stewardship_2025_chunk AS
SELECT
    relative_path,
    scoped_file_url,
    SPLIT_PART(relative_path, '/', -1) AS file_name,
    c.index::INT AS chunk_index,
    c.value:chunk::STRING AS chunk_text,
    c.value:headers:header_1::STRING AS header_1,
    c.value:headers:header_2::STRING AS header_2,
    c.value:headers:header_3::STRING AS header_3,
    MD5_HEX(relative_path || ':' || c.index)::STRING AS chunk_id
FROM gpif_stewardship_2025 t,
LATERAL FLATTEN(
    INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_MARKDOWN_HEADER(
        t.raw_text,
        OBJECT_CONSTRUCT('#', 'header_1', '##', 'header_2', '###', 'header_3'),
        10000
    )
) c;

In [ ]:
-- 結果確認（ヘッダー情報も含めて）
SELECT chunk_index, header_1, header_2, LEFT(chunk_text, 200) AS chunk_preview
FROM gpif_stewardship_2025_chunk ORDER BY chunk_index;

## Step 2-3: GPIFサステナビリティレポートの処理

[ソース](https://www.gpif.go.jp/en/investment/esg/gpif_publishes_the_2024_sustainability_investment_report.html)データ

![img](https://lh3.googleusercontent.com/pw/AP1GczPJXNcfLz3oxY1Ge1eA2YN2J1iHgBN9jHz83rt1WJMCBxei0ULoU-NeBvnrxSlvkocZRxBZODi8tKjdXXbVmF_AT7Qy-rp-Txzx_Hwb29j-HkbQa88ZB-4-jBkd4RCkHLEyVIs4sm3sWwzTWGEslM74=w1309-h924-s-no-gm?authuser=0)

In [ ]:
-- PDFからテキストを抽出
CREATE OR REPLACE TABLE gpif_sustainability_report AS
SELECT
    relative_path,
    GET_PRESIGNED_URL('@demo_db.demo_sustainability.document_stage', relative_path) AS scoped_file_url,
    AI_PARSE_DOCUMENT(
        TO_FILE('@demo_db.demo_sustainability.document_stage', relative_path), 
        {'mode': 'LAYOUT', 'page_split': true}
    ) AS raw_text_dict
FROM DIRECTORY('@demo_db.demo_sustainability.document_stage')
WHERE relative_path LIKE 'gpif_esg_report/%.pdf';

In [ ]:
-- チャンク化
CREATE OR REPLACE TABLE gpif_sustainability_report_chunk AS
SELECT
    t.relative_path,
    t.scoped_file_url,
    SPLIT_PART(t.relative_path, '/', -1) AS file_name,
    p.value:index::INT AS page_index, 
    c.index::INT AS chunk_index_on_page,
    c.value::STRING AS chunk_text,
    MD5_HEX(t.relative_path || ':' || p.value:index::INT || ':' || c.index::INT)::STRING AS chunk_id
FROM 
    gpif_sustainability_report t,
    LATERAL FLATTEN(input => t.raw_text_dict:pages) p,
    LATERAL FLATTEN(
        INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            p.value:content::STRING, 'markdown', 1000, 100
        )
    ) c;

In [ ]:
select * from gpif_sustainability_report_chunk;

## Step 2-4: 海外年金基金サステナビリティレポートの処理

CalPERSの[Sustainable Investments 2030 Strategy](https://www.calpers.ca.gov/documents/202311-invest-item06d-01-a/download) を利用

![img](https://lh3.googleusercontent.com/pw/AP1GczOxpIPdpajVvziVCa7fww1wovacLTyzwkgWY8WzDY4qX7fSZHa1sTYqA5dsCU0AIxS1SLXKjKJlS_-XedGhdkKHyaH4x2cK2WkksZP2B27eu3-J0mOdZgazLcuEflQTKKZMD_OPivyqPBrztoUqHtud=w1272-h712-s-no-gm?authuser=0)

In [ ]:
-- PDFからテキストを抽出
CREATE OR REPLACE TABLE global_pf_sustainability_report AS
SELECT
    relative_path,
    GET_PRESIGNED_URL('@demo_db.demo_sustainability.document_stage', relative_path) AS scoped_file_url,
    AI_PARSE_DOCUMENT(
        TO_FILE('@demo_db.demo_sustainability.document_stage', relative_path), 
        {'mode': 'LAYOUT', 'page_split': true}
    ) AS raw_text_dict
FROM DIRECTORY('@demo_db.demo_sustainability.document_stage')
WHERE relative_path LIKE 'global_pf_esg_report/%.pdf';

In [ ]:
-- チャンク化
CREATE OR REPLACE TABLE global_pf_sustainability_report_chunk AS
SELECT
    t.relative_path,
    t.scoped_file_url,
    SPLIT_PART(t.relative_path, '/', -1) AS file_name,
    p.value:index::INT AS page_index, 
    c.index::INT AS chunk_index_on_page,
    c.value::STRING AS chunk_text,
    MD5_HEX(t.relative_path || ':' || p.value:index::INT || ':' || c.index::INT)::STRING AS chunk_id
FROM 
    global_pf_sustainability_report t,
    LATERAL FLATTEN(input => t.raw_text_dict:pages) p,
    LATERAL FLATTEN(
        INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            p.value:content::STRING, 'markdown', 1000, 100
        )
    ) c;

In [ ]:
select * from global_pf_sustainability_report_chunk;

## Step 2-5: 統合ビューの作成

複数のチャンクテーブルを統合するビューを作成します。

![img](https://lh3.googleusercontent.com/pw/AP1GczPZcCvHPhKuINu1hPD1pODZZWdQ3KW_MzPeQVv6rO0vmnlNcp9K8tir3r7YZJtEcd4aqeypQR5BRU_coUjQMG0er-H57QGJvfJhtoIBj3iHQLRGwIJYFrXcsPaLCsmO-94v-qX7qSP59U42diYfvaVK=w1637-h917-s-no-gm?authuser=0)

In [ ]:
-- 運用機関 + スチュワードシップ原則の統合ビュー
CREATE OR REPLACE VIEW combined_sustainability_chunks_view AS
SELECT 'GPIF' AS source_table, relative_path, scoped_file_url, file_name, chunk_text, chunk_id,
    NULL AS page_index, chunk_index AS chunk_index_in_file, NULL AS chunk_index_on_page
FROM gpif_stewardship_2025_chunk
UNION ALL
SELECT 'AM' AS source_table, relative_path, scoped_file_url, file_name, chunk_text, chunk_id,
    page_index, NULL AS chunk_index_in_file, chunk_index_on_page
FROM am_sustainability_report_chunk;

In [ ]:
select * from combined_sustainability_chunks_view;

In [ ]:
-- グローバル年金基金の統合ビュー
CREATE OR REPLACE VIEW combined_global_sustainability_view AS
SELECT 'GPIF_Sustainability' AS source_report, relative_path, scoped_file_url, file_name,
    page_index, chunk_index_on_page, chunk_text, chunk_id
FROM gpif_sustainability_report_chunk
UNION ALL
SELECT 'Global_PF_Sustainability' AS source_report, relative_path, scoped_file_url, file_name,
    page_index, chunk_index_on_page, chunk_text, chunk_id
FROM global_pf_sustainability_report_chunk;

In [ ]:
select * from combined_global_sustainability_view;

In [ ]:
-- データ件数の確認
SELECT 'am_sustainability_report_chunk' AS table_name, COUNT(*) AS row_count FROM am_sustainability_report_chunk
UNION ALL SELECT 'gpif_stewardship_2025_chunk', COUNT(*) FROM gpif_stewardship_2025_chunk
UNION ALL SELECT 'gpif_sustainability_report_chunk', COUNT(*) FROM gpif_sustainability_report_chunk
UNION ALL SELECT 'global_pf_sustainability_report_chunk', COUNT(*) FROM global_pf_sustainability_report_chunk;

# セッション3: Cortex Search の設定

## Cortex Searchとは？
Snowflake上でセマンティック検索（意味に基づく検索）を実現するマネージドサービスです。

**主な特徴:**
- 自動ベクトル化
- ハイブリッド検索
- 自動更新（TARGET_LAG）

![img](https://lh3.googleusercontent.com/pw/AP1GczNia2N9Du--fyT02eSJSDY9qwnHpMINlB0ga1sVpCo82O7CHFKzBOriM0s2OJynA5E1Ar7HEo1fpXXOVpDDwk-EDo_Akp0ln82z9aWYKJIOYjTigBssKRFdwpRlMghT5mmYGwOhf7jvGbvUSQTRzvfp=w1637-h917-s-no-gm?authuser=0)

In [ ]:
-- スチュワードシップ原則評価用の検索サービス
CREATE OR REPLACE CORTEX SEARCH SERVICE sustainability_report
    ON chunk_text
    ATTRIBUTES relative_path, scoped_file_url, file_name, page_index, chunk_index_on_page
    WAREHOUSE = HANDSON_WH
    TARGET_LAG = '1 hour'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT 
        chunk_text, 
        relative_path, 
        scoped_file_url, 
        file_name, 
        page_index, 
        chunk_index_on_page
    FROM combined_sustainability_chunks_view
);

In [ ]:
-- グローバル年金分析用の検索サービス
CREATE OR REPLACE CORTEX SEARCH SERVICE global_pf_sustainability_report
    ON chunk_text
    ATTRIBUTES relative_path, scoped_file_url, file_name, page_index, chunk_index_on_page, source_report
    WAREHOUSE = HANDSON_WH
    TARGET_LAG = '1 hour'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT 
        chunk_text, 
        relative_path, 
        scoped_file_url, 
        file_name, 
        page_index, 
        chunk_index_on_page, 
        source_report
    FROM combined_global_sustainability_view
);

アカウント内で作成された Cortex Search サービスを確認

In [ ]:
-- Cortex Search Service 確認
SHOW CORTEX SEARCH SERVICES;

SNOWFLAKE.CORTEX.SEARCH_PREVIEW 関数で単発クエリの結果をプレビューする
- ワークシートや Snowflake ノートブックから 単発クエリの結果をプレビューする用途の関数

In [ ]:
-- 検索テスト
SELECT PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT',
        '{
            "query": "GPIFのスチュワードシップ活動原則の内容の中で、議決権行使の内容はなんですか？",
            "columns": ["chunk_text", "file_name"],
            "limit": 5
        }'
    )
)['results'] as results;

In [ ]:
SELECT 
    '議決権行使の内容' as search_type,
    (results.index + 1) as result_rank,
    results.value['file_name']::string as document_title,
    results.value['chunk_text']::string as content,
    LENGTH(results.value['chunk_text']::string) as content_length,
    -- results.value['file_url']::string as source_file_url
FROM (
    SELECT 
        PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT',
                '{
                    "query": "GPIFのスチュワードシップ活動原則の内容の中で、議決権行使の内容はなんですか？",
                    "columns": ["chunk_text", "file_name"],
                    "limit": 5
                }'
            )
        )['results'] as results_array
),
LATERAL FLATTEN(input => results_array) as results
ORDER BY results.index;

### Cortex Search ダウンロードリンク生成ツール
- Cortex Searchで検索したドキュメントのダウンロードリンクを生成するストアドプロシージャを用意
- 最終的にSnowflake Intelligenceを使ってソースとなるドキュメントのダウンロードが可能に

In [ ]:
CREATE OR REPLACE PROCEDURE get_file_presigned_url_sp(
RELATIVE_FILE_PATH STRING, 
EXPIRATION_MINS INTEGER DEFAULT 60
)
RETURNS STRING
LANGUAGE SQL
COMMENT = '@demo_db.demo_sustainability.document_stage内のファイルに対するpresigned URLを生成します。入力は相対ファイルパスです。'
EXECUTE AS CALLER
AS
$$
DECLARE
    presigned_url STRING;
    sql_stmt STRING;
    expiration_seconds INTEGER;
    stage_name STRING DEFAULT '@demo_db.demo_sustainability.document_stage';
BEGIN
    expiration_seconds := EXPIRATION_MINS * 60;
    
    sql_stmt := 'SELECT GET_PRESIGNED_URL(' || stage_name || ', ' || '''' || RELATIVE_FILE_PATH || '''' || ', ' || expiration_seconds || ') AS url';

    EXECUTE IMMEDIATE :sql_stmt;

    SELECT "URL"
    INTO :presigned_url
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

    RETURN :presigned_url;
END;
$$;

In [ ]:
CALL get_file_presigned_url_sp('gpif_20250331_stewardship_activity_principle.pdf', 60)


# セッション3: Cortex Agent の設定

## Cortex Agentとは？
Snowflake上でAIエージェントを構築するためのサービスです。
LLMと外部ツール（Cortex Search等）を組み合わせて、複雑なタスクを自動化できます。

**参考**: https://docs.snowflake.com/en/sql-reference/sql/create-agent

![img](https://lh3.googleusercontent.com/pw/AP1GczPjiPg43Rl268WvYPOlDpLIHZ7j5Ya2NWfVWDc0C6jghA2Eh24STRlm_ah5aBHMU-fLO5QaRwN1BMN-Ix9BWMapwQ6ookc5zGWef3dzcdyWT9Pm7EdCmFR1Rb7z529DEZdYAP6meRNicROmpKqGOgp3=w1637-h919-s-no-gm?authuser=0)


In [ ]:
-- スチュワードシップ原則評価用のAIエージェント
CREATE OR REPLACE AGENT STEWARDSHIP_AGENT
WITH PROFILE='{"display_name": "スチュワードシップ評価アシスタント"}'
    COMMENT=$$ GPIFスチュワードシップ活動原則に基づき、運用機関のサステナビリティレポートを分析・評価するエージェントです。 $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたはスチュワードシップ活動原則の専門家です。運用機関のサステナビリティレポートを分析し、5つの原則への対応状況を評価します。\n\n【スチュワードシップ活動原則】\n原則1: 運用受託機関におけるコーポレート・ガバナンス体制\n原則2: 運用受託機関における利益相反管理\n原則3: エンゲージメントを含むスチュワードシップ活動方針\n原則4: 投資におけるESGなどのサステナビリティの考慮\n原則5: 議決権行使\n\n【回答のルール】\n1. 必ず検索ツールを使用して関連情報を取得してください\n2. 回答には必ず出典（ファイル名、ページ番号）を明記してください\n3. 原則の趣旨に沿った取り組みが確認できれば「対応している」と評価してください\n4. 情報が見つからない場合のみ「情報なし」と回答してください\n5. 日本語で回答してください\n6. 可能であれば参照ドキュメントのダウンロードリンクを提供してください",
    "orchestration": "運用機関のレポートに関する質問には、SustainabilitySearchツールを使用して関連情報を検索し、その結果に基づいて回答を生成します。検索結果のドキュメントについてダウンロードリンクを生成する場合は、DocURLToolを使用してください。",
    "sample_questions": [
      {
        "question": "各運用機関スチュワードシップ責任に関する方針を教えてください"
      },
      {
        "question": "各運用機関の利益相反管理の取り組みを比較してください"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "SustainabilitySearch",
        "description": "運用機関のサステナビリティレポートとスチュワードシップ活動原則を検索します。検索結果にはファイル名(file_name)、相対パス(relative_path)、ページ番号(page_index)などの情報が含まれます。"
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "DocURLTool",
        "description": "このツールは、参照ドキュメント用のCortex Searchツールから取得したrelative_pathを使用し、ユーザーがドキュメントを表示・ダウンロードするための一時URLを返します。\n\n返されたURLは、ドキュメントタイトルをテキストとし、このツールの出力をURLとするHTMLハイパーリンクとして表示する必要があります。",
        "input_schema": {
          "type": "object",
          "properties": {
            "expiration_mins": {
              "description": "URLの有効期限（分）。デフォルトは5にしてください。",
              "type": "number"
            },
            "relative_file_path": {
              "description": "Cortex Searchツールから取得されるrelative_pathの値です。",
              "type": "string"
            }
          },
          "required": [
            "expiration_mins",
            "relative_file_path"
          ]
        }
      }
    }
  ],
  "tool_resources": {
    "SustainabilitySearch": {
      "name": "DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT",
      "max_results": 12,
      "title_column": "RELATIVE_PATH",
      "id_column": "SCOPED_FILE_URL"
    },
    "DocURLTool": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "HANDSON_WH"
      },
      "identifier": "DEMO_DB.DEMO_SUSTAINABILITY.GET_FILE_PRESIGNED_URL_SP",
      "name": "GET_FILE_PRESIGNED_URL_SP(VARCHAR, DEFAULT NUMBER)",
      "type": "procedure"
    }
  }
}
$$;


In [ ]:
-- Agent確認
SHOW AGENTS;


# 🎉 ハンズオン完了！

## 作成したリソースまとめ

| カテゴリ | リソース名 | 説明 |
|---------|-----------|------|
| **テーブル** | `am_sustainability_report_chunk` | 運用機関レポートのチャンク |
| | `gpif_stewardship_2025_chunk` | スチュワードシップ原則のチャンク |
| | `gpif_sustainability_report_chunk` | GPIF ESGレポートのチャンク |
| | `global_pf_sustainability_report_chunk` | 海外年金基金レポートのチャンク |
| **ビュー** | `combined_sustainability_chunks_view` | スチュワードシップ評価用統合ビュー |
| | `combined_global_sustainability_view` | グローバル分析用統合ビュー |
| **Cortex Search** | `sustainability_report` | スチュワードシップ評価用検索 |
| | `global_pf_sustainability_report` | グローバル分析用検索 |
| **Cortex Agent** | `STEWARDSHIP_AGENT` | スチュワードシップ評価エージェント |

## 次のステップ
- Streamlitアプリでこれらのリソースを活用
- Cortex Agentとの対話によるレポート分析
